# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part II: Data Cleaning

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

ModuleNotFoundError: No module named 'openpyxl'

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


***

## Import Preprocessed Datasets

In [4]:
# Read Items_List.csv
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [5]:
# Display first 5 rows
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.0,ea,1.0,lb,SPICES
1,I-43851,AJWAIN 5LB,5.0,lb,454.0,g,SPICES
2,I-3617,ALLSPICE GROUND SHAKER,8.0,each,454.0,g,SPICES
3,I-1852,ALMOND BEV - ORIGINAL,24.0,CT,1.0,CT,BEVERAGE
4,I-44059,ALMOND BUTTER*,10.0,Kg,1.0,Kg,FOOD - GROCERY


In [6]:
# There are 486 rows and 7 columns
Items.shape

(676, 7)

In [7]:
# Read Ingredients_List.csv
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [8]:
# Display first 5 rows
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-9857,5.0,Kg,1.0,1.1111,P-10112
1,P-48933,1.0,Kg,1.0,1.0000,P-10113
2,I-4626,16.0,CT,1.0,2.6667,P-10115
3,P-9755,1.0,lb,1.0,1.0000,P-10119
4,P-18746,1.0,Kg,1.0,1.0000,P-10241


In [9]:
# There are 3278 rows and 6 columns
Ingredients.shape

(5497, 6)

In [10]:
# Read Preps_List.csv
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [11]:
# Display first 5 rows
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-68207,24|BATCH|Poblano Pork Stew*,100.0,PTN,PREP
1,P-61322,AIOLI|Pesto,4.0,L,ZDONT USE OK - PREP
2,P-46066,Almond Milk Yeild,900.0,ml,NaN
3,P-8814,Aoli- RoastedGarlic Thyme,1.1,L,NaN
4,P-49605,APPLE|Cubed,75.0,g,NaN


In [12]:
# There are 546 rows and 5 columns
Preps.shape

(903, 5)

In [13]:
# Read Product_List.csv that was created from 1_data preprocessing
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [14]:
# Display first 5 rows
Products.head()

,ProdId,Description,SalesGroup
0,R-73669,DEL|smoothie| mango pineapple,GV - DELISH
1,R-56613,DEL|Smoothie|Almonds forever,GV - DELISH
2,R-44306,DEL|Smoothie|Banana Matcha,GV - DELISH
3,R-43697,DEL|Smoothie|Blueberry Boost,GV - DELISH
4,R-46074,DEL|Smoothie|Bna Almnd Buta,GV - DELISH


In [15]:
# There are 223 rows and 3 columns
Products.shape

(402, 3)

In [16]:
# Read Coversions_List.csv that was created from 1_data preprocessing
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [17]:
# Display first 5 rows
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L


In [18]:
# There are 270 rows and 6 columns
Conversions.shape

(334, 6)

***
## Update Conversion List

In [19]:
# Add the specific conversion info from the newly-processed data to a unit conversion database
Update_Conv = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Conv_UpdateConv.csv"))
Update_Conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398,g
1,I-28697,0.005181,1.0,ea,193.000,g
2,I-47441,0.005181,1.0,ea,193.000,g
3,I-1905,0.012500,1.0,CT,80.000,g
4,I-47440,0.005051,1.0,ea,198.000,g
...,...,...,...,...,...,...
627,I-1119p,1.000000,1.0,CT,0.000,g
628,I-1004p,1.000000,1.0,CT,0.000,g
629,I-1126p,1.000000,1.0,CT,0.000,g
630,I-25121,1.000000,1.0,ea,0.000,g


In [20]:
# return dataframe with null values
# There are no null values
subset_conv = Update_Conv[Update_Conv["Multiplier"].isna()]
subset_conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [21]:
# takes a data frame and assigns a new column called Multiplier to the dataframe
# uses iterrows() to iterate through the rows.Then subset_conv will have a new column named Multiplier with the computed values.
def assign_multiplier(df):
    for ind, row in df.iterrows():
        if row["ConvertFromQty"] == 0 or row["ConvertToQty"] == 0:
            df.loc[ind, "Multiplier"] = 1
        else:
            df.loc[ind, "Multiplier"] = row["ConvertFromQty"] / row["ConvertToQty"]
        
assign_multiplier(subset_conv)

# May 8th change: We want to 
assign_multiplier(Update_Conv)
#subset_conv

In [22]:
# convert dataframe to csv file
Update_Conv = pd.concat([Update_Conv, subset_conv], axis=0)
Update_Conv.to_csv("data/cleaning/update/Conv_UpdateConv.csv", index=False)

In [23]:
for index, row in Update_Conv.iterrows():
    Id = row['ConversionId'] 
    if Id in Conversions['ConversionId'].values:
        Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace=True)
    else:
        print(f"Warning: 'ConversionId' {Id} not found in Conversions DataFrame. Skipping drop operation.")

In [24]:
frames = [Conversions, Update_Conv]
Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [25]:
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L
...,...,...,...,...,...,...
959,I-1119p,1.000000,1.0,CT,0.00,g
960,I-1004p,1.000000,1.0,CT,0.00,g
961,I-1126p,1.000000,1.0,CT,0.00,g
962,I-25121,1.000000,1.0,ea,0.00,g


In [26]:
# Convert the DataFrame: Conversions into a csv file called Conversions_Added.csv
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
Conversions.to_csv(path, index = False, header = True)

### Create Unit Converter

In [27]:
# Import standard unit conversion information and construct a dataframe
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [28]:
# Seperate uoms that converted to 'ml' or 'g'
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [29]:
# Construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [30]:
# Convert pounds to grams
std_converter(0.25,'lb')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)


(113.398, 'g')

In [31]:
# Test the std_converter
assert std_converter(0.25,'lb') == (113.398, 'g')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)


In [32]:
# Construct a unit converter for specific ingredients

# After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# Comments for spc_converter:
# The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# convert the quantity and UOM to a standardized unit.

# If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [33]:
# spc_cov2 is the same as spc_cov but without null values labelled as "nan" in the list
import math
spc_cov2 = [item for item in spc_cov if not(pd.isnull(item)) == True]
spc_cov2

['I-1122',
 'I-1197',
 'I-1274',
 'I-1759',
 'I-1759',
 'I-1813',
 'I-1813',
 'I-1814',
 'I-1814',
 'I-1820',
 'I-1820',
 'I-1821',
 'I-1821',
 'I-2158',
 'I-2159',
 'I-2159',
 'I-2159',
 'I-2174',
 'I-2188',
 'I-2208',
 'I-2208',
 'I-2217',
 'I-2217',
 'I-2220',
 'I-2220',
 'I-2220',
 'I-2251',
 'I-2251',
 'I-2251',
 'I-2254',
 'I-2254',
 'I-2306',
 'I-2310',
 'I-2596',
 'I-2866',
 'I-2949',
 'I-3080',
 'I-3113',
 'I-3113',
 'I-3113',
 'I-3141',
 'I-3143',
 'I-3143',
 'I-3144',
 'I-3144',
 'I-3144',
 'I-3145',
 'I-3145',
 'I-3146',
 'I-3146',
 'I-3149',
 'I-3159',
 'I-3223',
 'I-3223',
 'I-3224',
 'I-3224',
 'I-3225',
 'I-3225',
 'I-3284',
 'I-3284',
 'I-3284',
 'I-3321',
 'I-3348',
 'I-3365',
 'I-3365',
 'I-3367',
 'I-3370',
 'I-3386',
 'I-3386',
 'I-3386',
 'I-3387',
 'I-3387',
 'I-3387',
 'I-3387',
 'I-3388',
 'I-3388',
 'I-3388',
 'I-3389',
 'I-3389',
 'I-3389',
 'I-3390',
 'I-3390',
 'I-3390',
 'I-3391',
 'I-3391',
 'I-3391',
 'I-3392',
 'I-3392',
 'I-3392',
 'I-3435',
 'I-3435',

In [34]:
# Here we pass in the ingredient (I-1120) the quantity of the ingredient (1) and the unit of measurement: CT -> count
# This gives us the number of grams of that ingredient
spc_converter('I-1120', 1, 'CT')

(1, 'CT')

In [35]:
Conversions.loc[Conversions["ConversionId"] == "I-14190"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
863,I-14190,0.002,1.0,LOAF,500.0,g


In [36]:
c_list = Conversions["ConversionId"].unique()
"I-68700" in c_list

True

In [37]:
spc_converter("I-14190", 1, "LOAF")

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/1590069188.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)/float(multiplier)


(500.0, 'g')

In [38]:
spc_converter('I-47530', 7, 'ea')

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/1590069188.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)/float(multiplier)


(210.0, 'g')

***
## Items with Non-standard Units

In [39]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-9857,5.0,Kg,1.000,1.1111,P-10112
1,P-48933,1.0,Kg,1.000,1.0000,P-10113
2,I-4626,16.0,CT,1.000,2.6667,P-10115
3,P-9755,1.0,lb,1.000,1.0000,P-10119
4,P-18746,1.0,Kg,1.000,1.0000,P-10241
...,...,...,...,...,...,...
5492,I-22493,5.0,g,0.001,1.0000,R-73668
5493,P-26514,2.0,g,1.000,1.0000,R-73668
5494,P-44913,90.0,g,0.001,1.0000,R-73668
5495,P-50725,1.0,ea,1.000,1.0000,R-73668


In [40]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.0,ea,1.00,lb,SPICES
1,I-43851,AJWAIN 5LB,5.0,lb,454.00,g,SPICES
2,I-3617,ALLSPICE GROUND SHAKER,8.0,each,454.00,g,SPICES
3,I-1852,ALMOND BEV - ORIGINAL,24.0,CT,1.00,CT,BEVERAGE
4,I-44059,ALMOND BUTTER*,10.0,Kg,1.00,Kg,FOOD - GROCERY
...,...,...,...,...,...,...,...
671,I-6026,SALT KOSHER*,9.0,pak,1360.78,g,SPICES
672,I-12176,SALT SEA*COARSE KOSHER,1.0,CT,2.27,Kg,SPICES
673,I-4904,STRAWBERRIES (CLAMSHELL) US,8.0,lb,1.00,lb,PRODUCE
674,I-3592,SUNFLOWER RAW*TFC,2.0,bag,1.50,Kg,FOOD - GROCERY


In [41]:
# Filter out the items whose unit information is unknown 

# We find the column names
col_names = list(Ingredients.columns.values)

# Create a Items_Nonstd list
Items_Nonstd = []

# If the unit of measurement is not grams or ml and ingredient id starts with I and the ingredient is not in ConversionId column of Conversions 
# then we add it to Items_Nonstd list
for index, row in Ingredients.iterrows():
    Ingre = Ingredients.loc[index,'IngredientId']
    Uom = Ingredients.loc[index,'Uom']
    if Uom not in ['g', 'ml'] and Uom not in liquid_unit + solid_unit and Ingre.startswith('I') and Ingre not in Conversions["ConversionId"].tolist():
        Dict = {}
        Dict.update(dict(row))
        Items_Nonstd.append(Dict)

# Create a DataFrame from Items_Nonstd list
Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
# Remove duplicate ingredients of the same properties so that Items_Nonstd has only unique rows. 
Items_Nonstd.drop_duplicates(subset=['IngredientId'], inplace=True,)
Items_Nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [42]:
# Assigning a Description column to the Items_Nonstd    
for index, row in Items_Nonstd.iterrows():
    idx = row['IngredientId']
    filtered_items = Items.loc[Items['ItemId'] == idx, 'Description']
    if not filtered_items.empty:
        descrp = filtered_items.values[0]
        Items_Nonstd.loc[index, 'Description'] = descrp
    else:
        pass

In [43]:
Items_Nonstd.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [44]:
# Convert the Items_Nonstd DataFrame to a csv file
path = os.path.join(os.getcwd(), "data", "cleaning", "Items_Nonstd.csv")
Items_Nonstd.to_csv(path, index = False, header = True)

***
## Clean Preps Units

In [45]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-68207,24|BATCH|Poblano Pork Stew*,100.0,PTN,PREP
1,P-61322,AIOLI|Pesto,4.0,L,ZDONT USE OK - PREP
2,P-46066,Almond Milk Yeild,900.0,ml,NaN
3,P-8814,Aoli- RoastedGarlic Thyme,1.1,L,NaN
4,P-49605,APPLE|Cubed,75.0,g,NaN
...,...,...,...,...,...
898,P-54352,Garnish|Avocado Spinach Pasta,20.0,g,NaN
899,P-73667,PREP|Chili Crunch|SL,4.5,L,FEAST
900,P-73472,PREP|SALAD|Tomato CucMint|MED,6.0,Kg,NaN
901,P-70825,PREP|Sliced Strawberry*,1.3,Kg,NaN


In [46]:
pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.00,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.00,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.00,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.00,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.00,g
...,...,...,...,...,...,...,...
503,P-36563,G&G|Salad|Med Chickpea,1.0,ea,CK - FOOD,400.00,g
504,P-73973,HARVEST|Miso Steelhead,6.0,srvg,NaN,870.49,g
505,P-73969,HARVEST|Pear & Gorg Filo,12.0,ea,NaN,1430.00,g
506,P-73977,HARVEST|Squash Penne,6.0,srvg,NaN,1325.00,g


In [47]:
# Creates 2 new columns called StdQty and StdUom in the Preps DataFrame. These columns contain NaN values
# Preparing to fill in these columns with standardized quantities and units of measurement 
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [48]:
# Convert uom into 'g' or 'ml' for each prep using the unit converter

# Retrieve the PrepId, PakQty, and PakUOM from the current row
# Pass these values to spc_converter, then we update the StdQty and StdUom columns of the current row with the converted values.
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/4107885356.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PTN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/3899604546.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Qty = float(qty)*float(multiplier)
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_88079/3899604546.py:5: FutureWarning: Calling 

In [49]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-68207,24|BATCH|Poblano Pork Stew*,100.0,PTN,PREP,100.0,PTN
1,P-61322,AIOLI|Pesto,4.0,L,ZDONT USE OK - PREP,4000.0,ml
2,P-46066,Almond Milk Yeild,900.0,ml,NaN,900.0,ml
3,P-8814,Aoli- RoastedGarlic Thyme,1.1,L,NaN,1100.0,ml
4,P-49605,APPLE|Cubed,75.0,g,NaN,75.0,g
...,...,...,...,...,...,...,...
898,P-54352,Garnish|Avocado Spinach Pasta,20.0,g,NaN,20.0,g
899,P-73667,PREP|Chili Crunch|SL,4.5,L,FEAST,4500.0,ml
900,P-73472,PREP|SALAD|Tomato CucMint|MED,6.0,Kg,NaN,6000.0,g
901,P-70825,PREP|Sliced Strawberry*,1.3,Kg,NaN,1300.0,g


In [50]:
# Save cleaned preps list to file
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

In [51]:
pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.00,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.00,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.00,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.00,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.00,g
...,...,...,...,...,...,...,...
503,P-36563,G&G|Salad|Med Chickpea,1.0,ea,CK - FOOD,400.00,g
504,P-73973,HARVEST|Miso Steelhead,6.0,srvg,NaN,870.49,g
505,P-73969,HARVEST|Pear & Gorg Filo,12.0,ea,NaN,1430.00,g
506,P-73977,HARVEST|Squash Penne,6.0,srvg,NaN,1325.00,g


### Get Preps with Nonstandard Unit

In [52]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [53]:
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-68207,24|BATCH|Poblano Pork Stew*,100.00,PTN,PREP,100.00,PTN
1,P-70055,BASIC|Egg Omelette|AAD,1.00,each,NaN,1.00,each
2,P-50725,BREAD|Focaccia|Portion,32.00,ea,PREP,32.00,ea
3,P-7523,BREAD|Garlic bread country|1ea,12.00,ea,PREP,12.00,ea
4,P-62284,BREAD|Garlic cheezey toast|FLX,16.00,ea,NaN,16.00,ea
...,...,...,...,...,...,...,...
60,P-62067,YIELD|Gluten Free white loaf,12.00,slice,NaN,12.00,slice
61,P-50658,YIELD|Golden apple,0.75,CT,NaN,0.75,CT
62,P-64994,YIELD|Marble Rye Sandwich,15.00,slice,NaN,15.00,slice
63,P-64902,YIELD|Sourdough clubhouse 5/8,20.00,slice,NaN,20.00,slice


In [54]:
# Filter out preps with nonstandard uom but have information already
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))

col_names = list(Preps_Nonstd.columns.values)
Preps_Nonstd_na = []

for index, row in Preps_Nonstd.iterrows():
    PrepId = Preps_Nonstd.loc[index,'PrepId']
    if PrepId not in Manual_PrepU['PrepId'].values:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd_na.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd_na, columns = col_names)
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom


In [55]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

In [56]:
update_prep = pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")
update_prep

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.00,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.00,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.00,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.00,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.00,g
...,...,...,...,...,...,...,...
503,P-36563,G&G|Salad|Med Chickpea,1.0,ea,CK - FOOD,400.00,g
504,P-73973,HARVEST|Miso Steelhead,6.0,srvg,NaN,870.49,g
505,P-73969,HARVEST|Pear & Gorg Filo,12.0,ea,NaN,1430.00,g
506,P-73977,HARVEST|Squash Penne,6.0,srvg,NaN,1325.00,g


***

## New Items

In [57]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [58]:
Items_Assigned.shape

(2991, 9)

In [59]:
Items_Assigned["InventoryGroup"].unique()

array(['MEAT', 'DELI & PREPARED MEAT', 'FOOD - GROCERY', 'POULTRY',
       'PRODUCTION FOOD', 'DAIRY', 'BAKING-RAW INGREDIENTS', 'PRODUCE',
       'ICECREAM/NOVELTY/GELATO', 'SEAFOOD', 'CANDIES SNACKS', 'MISC.',
       'MM Grocery Cereal', 'BAKED GOODS', 'MM Grocery Chill', 'PREP',
       'BREAD', 'MM BAKERY', 'ZDONT USE FOOD', 'SPICES', 'FAIR TRADE',
       'MM Snack Ethnic', 'HM FROZEN SINGLE SERVE',
       'CHIPS, PRETZELS, NUTS', 'ZDONT USE FROZEN', 'BEVERAGE',
       'PACKAGED BEER, CIDER', 'LIQUOR', 'ALCOHOL - FOOD',
       'MM CANDY Chocolate ', 'PREGEL - ALL ITEMS',
       'HM SWEET & SAVOURY SNACKS', 'UCARE',
       'HM CHILL SINGLE SERVE BEVERAGE', 'JERKY, PEPPERONI', 'SUSHI',
       'MM CANDY Bulk', 'MM CANDY Gum & Mints', 'MM Snack Single Bars',
       'MERCHANDISE', 'MM Frozen Grocery', 'MM Frozen HMR', 'MM Grocery',
       'MM Grocery Coffee & Tea', 'MM Grocery Ethnic', 'MM Grocery Soup',
       'MM MERCH OTC', 'MM Snack Jerky', 'ZDONT USE CANDY - GUM & MINTS',
       'DI

In [60]:
Items_Assigned["CategoryID"].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
       28., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57.,
       58., 59., 60., 61., 63., 15.])

In [61]:
Items_Assigned[Items_Assigned["CategoryID"] == 55]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
1287,I-11794,55.0,YERBA MATE REVEL BERRY 458ML,12.0,each,1.0,each,BEVERAGE,NaN
1288,I-1683,55.0,WATER VITAMIN ZERO XOXO,12.0,591ML,1.0,591ML,BEVERAGE,NaN
1289,I-2955,55.0,WATER SPARKLING SAN PELLEG,24.0,PTN,1.0,PTN,BEVERAGE,NaN
1290,I-52773,55.0,WATER SMART SPARKLING RASPB,12.0,each,1.0,each,BEVERAGE,NaN
1291,I-5983,55.0,WATER FOR RECIPES,1.0,L,1.0,L,BEVERAGE,NaN
...,...,...,...,...,...,...,...,...,...
2818,I-45755,55.0,WATER ACTIVE ORANGE MANGO,12.0,450ML,1.0,450ML,BEVERAGE,NaN
2819,I-45756,55.0,WATER ACTIVE STRAWB BLK CHERRY,12.0,450ML,1.0,450ML,BEVERAGE,NaN
2820,I-43251,55.0,WATER FRUIT 500ML PINEAPPLE,12.0,500ML,1.0,bottle,BEVERAGE,NaN
2855,I-71665,55.0,ICE CUBES,1.0,L,1.0,L,BEVERAGE,manually adjusted


### Get the List of New Items

In [62]:
# Filter new items by itemID that are not in the database and output them in a dataframe
col_names = list(Items.columns.values)
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [63]:
New_Items.insert(1, "CategoryID", '')
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [64]:
New_Items.shape

(0, 8)

In [65]:
# Store the list of new items into .csv file
# If New_Items is not empty then we convert it to a csv file. 
if not New_Items.empty:
    path = os.path.join(os.getcwd(), "data", "mapping", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
    New_Items.to_csv(path, index = False, header = True)

In [66]:
# file = pd.read_csv("data/mapping/new items/2022-11-01_New_Items.csv")
# file.to_excel("2022-11-01_New_Items.xlsx",index = None, header=True)
# if not New_Items.empty:
#     file = pd.read_csv("C:/Users/ENTER_HERE/CFFS-S23/CFFS-22-23/data/mapping/new items/"+ str(datetime.date(datetime.now()))+"_New_Items.csv")
#     file.to_excel(str(datetime.date(datetime.now()))+"_New_Items.xlsx",index = None, header=True)

In [67]:
# file2 = pd.read_excel("data/mapping/new items added/New_Items_Added_11.xlsx")
# file2.to_csv("data/mapping/new items added/New_Items_Added_11.csv", index=False)

file2 = pd.read_excel("data/mapping/new items added/New_Items_2023/New_Items_Added_2023_08_01.xlsx")
file2.to_csv("data/mapping/new items added/New_Items_2023/New_Items_Added_2023_08_01.csv", index=False)

***
## Data Summary

In [68]:
datasum = pd.DataFrame([New_Items.shape, Preps_Nonstd.shape, Items_Nonstd.shape],
                       columns = ['count', 'columns'], 
                       index = ['New_Items', 'Preps_Nonstd', 'Items_Nonstd'])
datasum

,count,columns
New_Items,0,8
Preps_Nonstd,0,7
Items_Nonstd,0,6


In [69]:
print(New_Items.columns)

Index(['ItemId', 'CategoryID', 'Description', 'CaseQty', 'CaseUOM', 'PakQty',
       'PakUOM', 'InventoryGroup'],
      dtype='object')
